In [1]:
!pip install crewai pypdf langchain langchain_community openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [3]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [4]:
# === Load PDF and Split ===
def load_pdf_content(path):
    loader = PyPDFLoader(path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(docs)
    return "\n\n".join([c.page_content for c in chunks])

In [5]:
# == PDF Loading ===
pdf_text = load_pdf_content("/content/PDF_JSON.pdf")

In [6]:
# Setup LLM
openai_llm = LLM(
    model="gpt-4o"
)

In [7]:
# === Define Agent ===
json_agent = Agent(
    role="PDF Information Extractor",
    goal="Convert PDF data into structured JSON format",
    backstory="You're an expert in extracting structured data from messy documents.",
    llm=openai_llm,
    verbose=True
)

In [8]:
# === Define Task ===
json_task = Task(
    description=f"""Extract structured data from this text and return a valid JSON.
Text:
{pdf_text[:4000]}

Output must be:
- Strict JSON (no markdown, no backticks)
- Format based on content (e.g., name, date, department if available)
""",
    expected_output="Valid JSON structure with key-value pairs from the text.",
    agent=json_agent
)

In [9]:
# === Define Crew ===
json_crew = Crew(
    agents=[json_agent],
    tasks=[json_task],
    process=Process.sequential,
    verbose=True
)

In [10]:
# === Run Agent ===
json_result = json_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 670326da-e86f-46d4-8699-d33edca7ce4d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 94fe1740-488b-4cc2-9fc9-6766c54a168d
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 94fe1740-488b-4cc2-9fc9-6766c54a168d
       Status: Executing Task...
    └── 🤖 Agent: PDF Information Extractor
            Status: In Progress

# Agent: PDF Information Extractor
## Task: Extract structured data from this text and return a valid JSON.
Text:
Employee  Name:  Alice  Johnson    Position:  Software  Engineer    Joining  Date:  January  2021    Department:  Engineering     Employee  Name:  Brian  Smith    Position:  HR  Specialist    Joining  Date:  March  2020    Department:  Human  Resources     Employee  Name:  Catherine  Lee    Position:  Financial  Analyst    Joining  Date:  May  2019    Department:  Finance     Employee  Name:  David  Wright    Position:  Product  Manager    Joining  Date:  June  2022    Department:  Product     Employee  Name:  Emily  Davis    Position:  UX  Designer    Joining  Date:  February  2023    Department:  Design     Employee  Name:  Faisal  Khan    Position:  Data  Scientist    Joining  Date:  August  2021    Department:  AI  &  Analytics     Employee  Name:  Grace  Thomas    Position:  Legal  Advisor    Joining  Date:  October  2020    Department:  Legal     Employee  Name:  Henr

🚀 Crew: crew
└── 📋 Task: 94fe1740-488b-4cc2-9fc9-6766c54a168d
       Status: Executing Task...
    └── 🤖 Agent: PDF Information Extractor
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: PDF Information Extractor
## Final Answer: 
{
  "employees": [
    {
      "name": "Alice Johnson",
      "position": "Software Engineer",
      "joining_date": "January 2021",
      "department": "Engineering"
    },
    {
      "name": "Brian Smith",
      "position": "HR Specialist",
      "joining_date": "March 2020",
      "department": "Human Resources"
    },
    {
      "name": "Catherine Lee",
      "position": "Financial Analyst",
      "joining_date": "May 2019",
      "department": "Finance"
    },
    {
      "name": "David Wright",
      "position": "Product Manager",
      "joining_date": "June 2022",
      "department": "Product"
    },
    {
      "name": "Emily Davis",
      "position": "UX Designer",
      "joining_date": "February 2023",
      "department": "Design"
    },
    {
      "name": "Faisal Khan",
      "position": "Data Scientist",
      "joining_date": "August 2021",
      "department": "AI & Analytics"
    },
    {
      "name": "Grace Thomas

🚀 Crew: crew
└── 📋 Task: 94fe1740-488b-4cc2-9fc9-6766c54a168d
       Status: Executing Task...
    └── 🤖 Agent: PDF Information Extractor
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 94fe1740-488b-4cc2-9fc9-6766c54a168d
       Assigned to: PDF Information Extractor
       Status: ✅ Completed
    └── 🤖 Agent: PDF Information Extractor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 94fe1740-488b-4cc2-9fc9-6766c54a168d                                                                     │
│  Agent: PDF Information Extractor                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 670326da-e86f-46d4-8699-d33edca7ce4d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
print("\n JSON Output:\n", json_result)


 JSON Output:
 {
  "employees": [
    {
      "name": "Alice Johnson",
      "position": "Software Engineer",
      "joining_date": "January 2021",
      "department": "Engineering"
    },
    {
      "name": "Brian Smith",
      "position": "HR Specialist",
      "joining_date": "March 2020",
      "department": "Human Resources"
    },
    {
      "name": "Catherine Lee",
      "position": "Financial Analyst",
      "joining_date": "May 2019",
      "department": "Finance"
    },
    {
      "name": "David Wright",
      "position": "Product Manager",
      "joining_date": "June 2022",
      "department": "Product"
    },
    {
      "name": "Emily Davis",
      "position": "UX Designer",
      "joining_date": "February 2023",
      "department": "Design"
    },
    {
      "name": "Faisal Khan",
      "position": "Data Scientist",
      "joining_date": "August 2021",
      "department": "AI & Analytics"
    },
    {
      "name": "Grace Thomas",
      "position": "Legal Advisor",
 